In [1]:
import os
import pickle
import warnings
from typing import ( Optional )
from collections import defaultdict
from datetime import datetime, timedelta


# 경고 메시지 무시 설정
warnings.filterwarnings('ignore')

# 모델 로드
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# 기준연월 설정
# 모델학습했던 시점인 202304를 기준연월로 잡음 이 기준연월은 재학습을 하기 전까지 유지한다
# 202304을 기준연월로 변경하여 date_dictionary.txt 업데이트시 diff_month 값을 증가시켜 현재보다 미래시점의 value를 상승
standard_ym = '202304'

def getPredictY(
    x: int
) -> int:
    return int(loaded_model.predict([[x]]))

def getAllDates():
    today = datetime.today()
    start_date = datetime(1, 1, 1)
    end_date = datetime(today.year, 12, 31)
    all_dates = []

    current_date = start_date
    while current_date <= end_date:
        all_dates.append(current_date.strftime('%Y%m%d'))
        current_date += timedelta(days=1)

    return all_dates

def calculateMonthDifference(
    start_ym: str,
    end_ym: str
) -> int:
    # 입력된 날짜 문자열을 datetime 객체로 변환
    start_dt = datetime.strptime(start_ym, '%Y%m')
    end_dt = datetime.strptime(end_ym, '%Y%m')

    # 연월의 차이 계산
    month_difference = (end_dt.year - start_dt.year) * 12 + (end_dt.month - start_dt.month)

    return month_difference

def calculateNewYearMonth(
    ym: str,
    months: int
) -> Optional[str]:
    try:
        # 입력된 날짜 문자열을 datetime 객체로 변환
        ym = datetime.strptime(ym, '%Y%m')

        # 개월수 차이에 따라 연월을 재계산
        new_ym = ym - timedelta(days=months*30)

        # 재계산된 연월을 문자열로 변환하여 반환
        new_ym = new_ym.strftime('%Y%m')
        return new_ym
    except:
        return None

In [2]:
# 현재와 모델 학습시점간의 연월 차이를 계산
month_diff = calculateMonthDifference(
    standard_ym,
    f'{datetime.today().year}{str(datetime.today().month).zfill(2)}'
)

month_diff

9

In [3]:
# 현재와 모델 학습시점간의 연월 차이를 계산
month_diff = calculateMonthDifference(
    standard_ym,
    f'{datetime.today().year}{str(datetime.today().month).zfill(2)}'
)

# 오늘을 기준으로 올해 마지막날까지의 모든 날짜 리스트 생성
all_dates = getAllDates()

# key: 날짜, value: 빈도수를 갖는 딕셔너리 생성
# 기본값을 0으로 설정한 defaultdict 생성
all_dt_dic = defaultdict(int)

for dt in all_dates:
    ym = dt[:6]
    # 연월 차이로 재계산한 연월을 산출
    ym = calculateNewYearMonth(
        ym=ym,
        months=month_diff
    )
    if ym:
        pred_y = getPredictY(ym)
        if pred_y > 0:
            all_dt_dic[dt] = pred_y
        else:
            all_dt_dic[dt] = 0


In [4]:
def save_dictionary_to_txt(dictionary, filename):
    with open(filename, 'w') as file:
        for key, value in dictionary.items():
            line = f"{key}${value}\n"
            file.write(line)

# 딕셔너리를 텍스트 파일로 저장
notebook_path = os.getcwd()
print(notebook_path)
notebook_path = '/'.join(notebook_path.split('/')[:-1])
save_dictionary_to_txt(all_dt_dic, f'{notebook_path}/mjjo/data/date_dictionary.txt')


/Users/macrent/Documents/mjjo/vos-mjjo/model


In [5]:
all_dt_dic

defaultdict(int,
            {'00011001': 4,
             '00011002': 4,
             '00011003': 4,
             '00011004': 4,
             '00011005': 4,
             '00011006': 4,
             '00011007': 4,
             '00011008': 4,
             '00011009': 4,
             '00011010': 4,
             '00011011': 4,
             '00011012': 4,
             '00011013': 4,
             '00011014': 4,
             '00011015': 4,
             '00011016': 4,
             '00011017': 4,
             '00011018': 4,
             '00011019': 4,
             '00011020': 4,
             '00011021': 4,
             '00011022': 4,
             '00011023': 4,
             '00011024': 4,
             '00011025': 4,
             '00011026': 4,
             '00011027': 4,
             '00011028': 4,
             '00011029': 4,
             '00011030': 4,
             '00011031': 4,
             '00011101': 0,
             '00011102': 0,
             '00011103': 0,
             '00011104': 0,
   

In [6]:
all_dt_dic['10000101']

0

In [7]:
all_dt_dic['19800101']

310

In [8]:
all_dt_dic['20000101']

659

In [9]:
all_dt_dic['20100101']

1041

In [10]:
all_dt_dic['20200101']

1302

In [11]:
all_dt_dic['20220101']

1197

In [12]:
all_dt_dic['20230101']

874

In [13]:
all_dt_dic['20230701']

479

In [26]:
all_dt_dic['20231101']

312

In [15]:
all_dt_dic['20231201']

251

In [16]:
all_dt_dic['20240101']

0

In [19]:
all_dt_dic['20241231']

5

In [31]:
x = 202011

pred_y = int(loaded_model.predict([[x]]))
print(pred_y)

1098
